In [1]:
import sys
if '/scratch/zq415/grammar_cor/pose/pose_estimation_unified/src' in sys.path:
    sys.path.remove('/scratch/zq415/grammar_cor/pose/pose_estimation_unified/src')

import time
import torch
from options.train_options import TrainOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer


In [ ]:
device = torch.device('cuda:{}'.format(0))
netG = networks.define_G(3, 3, 64, 'resnet_9blocks', 'instance', False, 'normal', 0.02, [0])
state_dict = torch.load('./checkpoints/render2real_cyclegan2/latest_net_G_A.pth', map_location=str(device))
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if 'module.' not in k:
        k = 'module.'+ k
        new_state_dict[k]=v
        
netG.load_state_dict(new_state_dict)
netG.to(device)

In [ ]:
rendered_path = '/scratch/zq415/grammar_cor/pose/pose_estimation_unified/data/data_collection/robot_lab_03_13_2020_000/rendered_images'
render_img_paths, render_scr_paths, render_poses = make_render_dataset(rendered_path)
print(len(render_img_paths), len(render_scr_paths), render_poses.shape)

In [ ]:
train_dataset = dataset_scr(render_img_paths[:1000], render_poses, render_scr_paths,
                        transform=transforms.Compose([random_crop()]))
train_dataloader = DataLoader(train_dataset, batch_size=1,
                            shuffle=False, num_workers=4)

In [ ]:
netG.eval()
for i_batch, sample_batched in enumerate(train_dataloader):
    inputs, scrs, poses = (sample_batched['image']-0.5)/0.5, sample_batched['scr'], sample_batched['pose']
    labels = sample_batched['scr']
    inputs = inputs.to(device, dtype=torch.float)
    with torch.no_grad():
        fake_B = netG(inputs)
    save_image_test(fake_B, './results', str(i_batch)+'_fake_real')
    save_image_test(inputs, './results', str(i_batch)+'_render')